**Importing necessary depencencies**

In [1]:
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
!pip install dateparser
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 10.4 MB/s eta 0:00:00


In [2]:
action_verbs = [
    "must", "should", "might", "can", "could", "shall", "will", "would",
    "may", "ought", "dare", "let", "do", "does", "had better", "better",
    "needn't", "shall not", "shall"
] #These words clearly indicate a task in any paragraph. If there are additional words that signify a task, they can be included to enhance performance.
to_form_verbs = [
    "has", "have", "had", "used", "need", "needs", "ought", "required",
    "want", "wants", "wish", "wished", "supposed", "going", "bound",
    "determined", "expected", "forced", "obliged", "intended", "aim",
    "aims", "plan", "plans", "set", "decided", "attempt", "attempts",
    "fail", "fails", "manage", "manages", "struggle", "struggles"
] #When the word "to" follows these words, it indicates a task that needs to be completed.
stopwords_list = set(stopwords.words('english'))
be_form= ["be", "am", "is", "are", "was", "were", "being", "been",
         "have", "has", "had", "having"] #to remove these words from finding action words
pronouns=['he','she','it','they','we'] #pronouns to identify the person whom has to do the task

In [11]:

def preprocessing(sentences):

  #removing html tags from the input sentences
  tag_re = re.compile(r"<[^>]+>")
  sentences = tag_re.sub('', sentences)

  #making sentences in to splits of each sentences

  sents = sent_tokenize(sentences)
  sent_words = []
  for i in sents:
    a = list(i.split())
    sent_words.append(a)

  #converting the text into lower case and removing punctuations and numbers
  sentences = re.sub(r'[^a-zA-Z\s]', '', sentences)
  sentences = re.sub(r"\b[a-zA-Z]\b", '', sentences)

  #remove multiple spaces
  sentences = re.sub(r'\s+', ' ', sentences).strip()

  #spliiting the sentences into individual words
  words = sentences.split()
  pos_tags = pos_tag(words)

  #finding the action words in the sentences
  action_words = [word for word, tag in pos_tags if tag.startswith('VB') if word not in be_form ]

  #finding the subject from the data
  subject = [word for word, tag in pos_tags if tag.startswith('NN') if word.istitle()]

  #extracting the task from the input text
  for i in action_words:
    for j in range(len(words)):
      if words[j]==i and words[j-1] in action_verbs:
        print(subject[0],words[j-1],i," ".join(' '.join(sublist) for sublist in [sent_words[a][b+1:] for a in range(len(sent_words)) for b in range(len(sent_words[a])) if sent_words[a][b]==i]))
      elif words[j]==i and words[j-2] in to_form_verbs:
        print(subject[0],words[j-2],"to",i," ".join(' '.join(sublist) for sublist in [sent_words[a][b+1:] for a in range(len(sent_words)) for b in range(len(sent_words[a])) if sent_words[a][b]==i]))

In [4]:
import re
import dateparser

def extract_time_data(text):
    # Regex patterns for different time-related mentions
    day_pattern = r'\b(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\b'
    date_pattern = r'\b(\d{1,2})(?:st|nd|rd|th)?\s+of\s+\b(January|February|March|April|May|June|July|August|September|October|November|December)\b'
    time_pattern = r'\b(\d{1,2}(?::\d{2})?\s?(?:AM|PM|am|pm)?)\b'
    period_pattern = r'\b(morning|afternoon|evening|night|midnight|noon|now)\b'

    # Extract matches
    days = re.findall(day_pattern, text, re.IGNORECASE)
    dates = re.findall(date_pattern, text, re.IGNORECASE)
    times = re.findall(time_pattern, text, re.IGNORECASE)
    periods = re.findall(period_pattern, text, re.IGNORECASE)

    # Convert dates to full format
    formatted_dates = [dateparser.parse(f"{d[0]} {d[1]}").strftime('%d %B') for d in dates]

    return {
        "days": days,
        "dates": formatted_dates,
        "times": times,
        "periods": periods
    }

In [12]:
text="Rahul wakes up early every day. He goes to college in the morning and comes back at 3 pm. At present,<html> Rahul is outside. He has to buy the snacks for all of us. He is very good in his studies. He should clean the room before 5pm"
print(preprocessing(text))
print(extract_time_data(text))

Rahul has to buy the snacks for all of us.
Rahul should clean the room before 5pm
None
{'days': [], 'dates': [], 'times': ['3 pm', '5pm'], 'periods': ['morning']}


In [ ]:
text="Sophie starts her day with a cup of coffee and a morning walk. She spends a few hours attending online lectures. Before lunch, she has to submit her assignment on machine learning. In the afternoon, she takes a short nap and then helps her brother with his studies. In the evening, she enjoys watching her favorite TV series."
print(preprocessing(text))
print(extract_time_data(text))

Sophie has to submit her assignment on machine learning.
(['starts', 'spends', 'attending', 'submit', 'takes', 'helps', 'enjoys', 'watching'], ['Sophie'], [['Sophie', 'starts', 'her', 'day', 'with', 'a', 'cup', 'of', 'coffee', 'and', 'a', 'morning', 'walk.'], ['She', 'spends', 'a', 'few', 'hours', 'attending', 'online', 'lectures.'], ['Before', 'lunch,', 'she', 'has', 'to', 'submit', 'her', 'assignment', 'on', 'machine', 'learning.'], ['In', 'the', 'afternoon,', 'she', 'takes', 'a', 'short', 'nap', 'and', 'then', 'helps', 'her', 'brother', 'with', 'his', 'studies.'], ['In', 'the', 'evening,', 'she', 'enjoys', 'watching', 'her', 'favorite', 'TV', 'series.']])
{'days': [], 'dates': [], 'times': [], 'periods': ['morning', 'afternoon', 'evening']}


In [7]:
text="Ryan wakes up early every morning and goes for a jog in the park. After returning home, he takes a quick shower and has breakfast. He must complete his project report before heading to work. Later in the evening, he plans to meet his friends for dinner. He usually spends some time reading before going to bed."
print(preprocessing(text))
print(extract_time_data(text))

Ryan must complete his project report before heading to work.
Ryan plans to meet his friends for dinner.
Ryan going to bed 
(['wakes', 'goes', 'returning', 'takes', 'complete', 'heading', 'work', 'plans', 'meet', 'spends', 'reading', 'going', 'bed'], ['Ryan'], [['Ryan', 'wakes', 'up', 'early', 'every', 'morning', 'and', 'goes', 'for', 'a', 'jog', 'in', 'the', 'park.'], ['After', 'returning', 'home,', 'he', 'takes', 'a', 'quick', 'shower', 'and', 'has', 'breakfast.'], ['He', 'must', 'complete', 'his', 'project', 'report', 'before', 'heading', 'to', 'work.'], ['Later', 'in', 'the', 'evening,', 'he', 'plans', 'to', 'meet', 'his', 'friends', 'for', 'dinner.'], ['He', 'usually', 'spends', 'some', 'time', 'reading', 'before', 'going', 'to', 'bed.']])
{'days': [], 'dates': [], 'times': [], 'periods': ['morning', 'evening']}


In [10]:
text="Amit wakes up early every morning and goes for a jog in the park. He enjoys reading books and often visits the library in the evening. Today, he has to submit his project report before noon. After that, he must call his professor to discuss his research findings. In the afternoon, he needs to buy groceries for the week. His mother reminded him that he should pay the electricity bill before the due date. Later, Amit plans to meet his friends for a study session. Before going to bed, he has to complete his assignment and review notes for the upcoming exam."
print(preprocessing(text))
print(extract_time_data(text))

Amit has to submit his project report before noon.
Amit must call his professor to discuss his research findings.
Amit needs to buy groceries for the week.
Amit should pay the electricity bill before the due date.
Amit plans to meet his friends for a study session.
Amit going to bed 
Amit has to complete his assignment and review notes for the upcoming exam.
(['wakes', 'goes', 'enjoys', 'reading', 'visits', 'submit', 'call', 'discuss', 'needs', 'buy', 'reminded', 'pay', 'plans', 'meet', 'going', 'bed', 'complete'], ['Amit', 'Today', 'Later', 'Amit'], [['Amit', 'wakes', 'up', 'early', 'every', 'morning', 'and', 'goes', 'for', 'a', 'jog', 'in', 'the', 'park.'], ['He', 'enjoys', 'reading', 'books', 'and', 'often', 'visits', 'the', 'library', 'in', 'the', 'evening.'], ['Today,', 'he', 'has', 'to', 'submit', 'his', 'project', 'report', 'before', 'noon.'], ['After', 'that,', 'he', 'must', 'call', 'his', 'professor', 'to', 'discuss', 'his', 'research', 'findings.'], ['In', 'the', 'afternoon,'